In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Import the Data and Scale it

In [64]:
from numpy import genfromtxt
data = genfromtxt('./data/heston_data.csv', delimiter=',')
X_data = data[:,1:5]
Y_data = data[:,5:7]
print(X_data[:10,:])
print(Y_data[:10,:])

[[1.20207581 1.92239056 0.49583491 0.20336394]
 [1.57512242 1.44138507 0.90424946 0.18410172]
 [1.04617186 1.76098485 0.06558515 0.81524428]
 [1.43554335 1.87604335 0.68017867 0.34929233]
 [1.04482755 1.11180824 0.693482   0.42504613]
 [1.11737054 1.00940002 0.72709671 0.84580623]
 [1.44207922 1.38795771 0.78484338 0.6244721 ]
 [1.1884379  1.66271561 0.33986351 0.06144064]
 [1.77297475 1.81506917 0.36689373 0.98140273]
 [1.22890027 1.17113486 0.21459103 0.29242623]]
[[0.98416443 1.64142053]
 [1.00456183 1.35890458]
 [0.99018326 1.49340831]
 [1.00372842 1.67447324]
 [1.00327831 1.07150625]
 [0.97183347 0.9940148 ]
 [0.98204588 1.28494616]
 [1.0052962  1.46118496]
 [0.98318762 1.67426659]
 [0.98679588 1.11813847]]


In [65]:
from sklearn.preprocessing import MinMaxScaler

In [66]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_data)
Y_scaled = scaler.fit_transform(Y_data)

### Create Placeholders and Constants

In [92]:
num_inputs = X_data.shape[0]
num_outputs = Y_data.shape[0]
learning_rate = 0.001
num_steps = 2000
batch_size = 10
num_neurons = 4

X = tf.placeholder(tf.float32, [None, num_inputs])
y = tf.placeholder(tf.float32, [None, num_outputs])

In [88]:
#input: X,y,batch_size
#output: X_batch,y_batch: randomly chosen rows from X,y 
def next_batch(X,y,batch_size):
    max_int = X.shape[0]
    batch_ind = np.random.randint(0,max_int,size=batch_size)
    X_batch = X[batch_ind,:]
    y_batch = y[batch_ind,:]
    
    return X_batch,y_batch

In [76]:
from tensorflow.contrib.layers import fully_connected

#Layers
hidden1 = fully_connected(X, num_neurons, activation_fn=tf.nn.elu)
hidden2 = fully_connected(hidden1, num_neurons, activation_fn=tf.nn.elu)
outputs = fully_connected(hidden2, num_outputs, activation_fn=tf.nn.softmax)

In [77]:
#Loss Function
loss = tf.reduce_mean(tf.square(outputs - y))  # MSE

In [78]:
#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [79]:
init = tf.global_variables_initializer()

In [80]:
saver = tf.train.Saver()

In [91]:
with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(num_steps):
        
        X_batch,Y_batch = next_batch(X_scaled,Y_scaled,batch_size)
        sess.run(train,feed_dict={X: X_batch, y: Y_batch})
        
        if iteration % 100 == 0:
            
            mse = loss.eval(feed_dict={X: X_batch, y: Y_batch})
            print(iteration, "\tMSE:", mse)
    
    saver.save(sess, "./models/heston_nn")

0 	MSE: 0.102403685
100 	MSE: 0.059392273
200 	MSE: 0.045600016
300 	MSE: 0.060340516
400 	MSE: 0.07409399
500 	MSE: 0.063533306
600 	MSE: 0.04011544
700 	MSE: 0.08591964
800 	MSE: 0.058270417
900 	MSE: 0.034622453
1000 	MSE: 0.05068176
1100 	MSE: 0.020209536
1200 	MSE: 0.053438626
1300 	MSE: 0.037800763
1400 	MSE: 0.021396395
1500 	MSE: 0.041898724
1600 	MSE: 0.027298909
1700 	MSE: 0.020148799
1800 	MSE: 0.033910163
1900 	MSE: 0.035496168
